In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
from src.Graph import SingleGeneGraph
from src.Function import read_spatial_expression
from src import Preprocess
from tqdm import tqdm
%matplotlib inline

In [2]:
def spatial_plot(adata, color, map = 'bwr'):
    sc.set_figure_params(facecolor="white", figsize=(8, 8))
    ax=sc.pl.scatter(adata,alpha=1,x="array_row",y="array_col", color = color, 
                 title=color,show=False,size=500000/adata.shape[0],
                 color_map=map,frameon=False)
    ax.set_aspect('equal', 'box')
    # ax.axes.invert_yaxis()

In [3]:
kneighbors, beta = 4,2
dataDir = "../dataset/MOB-breast_cancer/Rep11_MOB_count_matrix-1.tsv"
locs, data, _ = read_spatial_expression(dataDir)
locsDf = pd.DataFrame(locs, columns=["imagerow", "imagecol"])
locsDf.index = data.index
adata = sc.AnnData(X=data, obs=locsDf)
adata = Preprocess.data_preprocess(adata)
new_index = ["array_row", "array_col"]
adata.obs.rename(
    columns={"imagerow": new_index[0], "imagecol": new_index[1]}, inplace=True
)


raw data dim: (262, 16218)
Number of expressed genes a spot must have to be kept (0.01% of total expressed genes) 3375
Marked 3 spots
Removing genes that are expressed in less than 3 spots with a count of at least 1
Dropped 1559 genes


In [4]:
exp = adata.to_df()
coord = adata.obs[['array_row', 'array_col']]
gene_id = 'Myo5a'

In [5]:
geneGraph = SingleGeneGraph(gene_id,exp, coord, kneighbors=kneighbors)

In [6]:
geneGraph.mrf_with_icmem(beta = beta)

 10%|█         | 1/10 [00:00<00:01,  8.60it/s]

Convergence reached
[[ 0.54259855  0.53660241]
 [-0.80407948  0.59440464]]


: 

In [4]:
adata.obs.index

Index(['16.92x9.015', '16.945x11.075', '16.97x10.118', '16.939x12.132',
       '16.949x13.055', '16.942x15.088', '16.941x17.065', '16.919x16.055',
       '9.862x9.024', '12.08x9.115',
       ...
       '20.089x24.006', '24.146x24.011', '23.124x24.046', '21.01x23.944',
       '22.053x23.962', '27.018x20.088', '17.964x10.137', '10.922x17.038',
       '18.006x20.069', '26.991x13.041'],
      dtype='object', length=262)

In [5]:
exp = adata.to_df()
exp.index

Index(['16.92x9.015', '16.945x11.075', '16.97x10.118', '16.939x12.132',
       '16.949x13.055', '16.942x15.088', '16.941x17.065', '16.919x16.055',
       '9.862x9.024', '12.08x9.115',
       ...
       '20.089x24.006', '24.146x24.011', '23.124x24.046', '21.01x23.944',
       '22.053x23.962', '27.018x20.088', '17.964x10.137', '10.922x17.038',
       '18.006x20.069', '26.991x13.041'],
      dtype='object', length=262)

In [6]:
exp 

Index(['Nrf1', 'Zbtb5', 'Ccnl1', 'Lrrfip1', 'Bbs1', 'Lix1', 'Whrn', 'Ate1',
       'Ubac1', 'Rab34',
       ...
       'Gm15910', 'Zc3h12a', 'Zcwpw1', '1700020D05Rik', '4930471G03Rik',
       '9030204H09Rik', 'Rn4.5s', 'Hist1h2ak', '4930562F07Rik', 'Gm20172'],
      dtype='object', length=14659)

In [3]:
exp = pd.read_csv("input/gsExp.csv")

In [5]:
exp.columns[1:]

Index(['AL645608.1', 'SAMD11', 'ISG15', 'TNFRSF18', 'SDF4', 'MXRA8', 'CCNL2',
       'VWA1', 'ATAD3B', 'FAAP20',
       ...
       'AP001468.1', 'LSS', 'MCM3AP-AS1', 'S100B', 'MT-ND1', 'MT-ND2',
       'MT-CO1', 'MT-ATP8', 'MT-ND3', 'MT-CYB'],
      dtype='object', length=3000)

In [7]:
coord = np.genfromtxt("input/gsCoord.csv")

array([nan, nan, nan, ..., nan, nan, nan])

In [8]:
adata = sc.read_visium("../dataset/goldStandard")

/home/qinxianhan/.conda/envs/sc/lib/python3.11/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/qinxianhan/.conda/envs/sc/lib/python3.11/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [10]:
coord = adata.obsm["spatial"]

In [12]:
np.savetxt("input/gsCoord.csv", coord, delimiter=",", fmt="%d")

In [3]:
gene_id = exp.columns[1]
kneighbors = 4
gene_id

'AC012360.3'

In [9]:
exp = pd.read_csv("input/gsExp.csv", index_col=0)
coord = np.genfromtxt("input/gsCoord.csv", delimiter=",")
geneGraph = SingleGeneGraph(gene_id,exp, coord, kneighbors=kneighbors)

In [5]:
geneGraph.mrf_with_icmem(beta = 2)

  0%|          | 0/10 [00:00<?, ?it/s]


IndexError: index 3636 is out of bounds for axis 0 with size 3636

In [10]:
geneGraph.graph

<3636x3636 sparse matrix of type '<class 'numpy.float64'>'
	with 14544 stored elements in Compressed Sparse Row format>

In [11]:
exp 

,IGKC,AC012360.3,MOBP,TMEM158,TF,CPB1,TMEM144,BACH2,NPY,MYO1G,...,IGHM,HBA2,HBA1,CNP,MYL9,PCSK4,PRR19,IGLC2,CR381653.1,LRRC3
AAACAAGTATCTCCCA-1,1.168548,-0.077593,-0.033543,-0.403004,-0.913560,-0.306615,-0.552796,-0.103783,-0.567012,-0.080026,...,-0.25909,0.433790,-0.439749,-0.624686,-0.362863,-0.119038,-0.099579,-0.431536,-0.02837,-0.044405
AAACAATCTACTAGCA-1,-0.668048,-0.077593,-0.779179,-0.403004,-0.913560,-0.306615,-0.552796,-0.103783,-0.567012,-0.080026,...,-0.25909,1.930715,-0.439749,0.363277,4.914625,-0.119038,-0.099579,-0.431536,-0.02837,-0.044405
AAACACCAATAACTGC-1,0.695366,-0.077593,2.062531,-0.403004,1.464825,-0.306615,1.204486,-0.103783,-0.567012,-0.080026,...,-0.25909,2.248626,-0.439749,1.858433,-0.362863,-0.119038,-0.099579,-0.431536,-0.02837,-0.044405
AAACAGAGCGACTCCT-1,-0.668048,-0.077593,0.695679,1.959511,0.020708,-0.306615,-0.552796,-0.103783,0.841627,-0.080026,...,-0.25909,0.772610,-0.439749,-1.286461,1.785889,-0.119038,-0.099579,1.159950,-0.02837,-0.044405
AAACAGCTTTCAGAAG-1,2.468108,-0.077593,0.372512,-0.403004,0.948639,-0.306615,-0.552796,-0.103783,-0.567012,-0.080026,...,-0.25909,-0.568908,-0.439749,0.478324,-0.362863,-0.119038,-0.099579,-0.431536,-0.02837,-0.044405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTTCACATCCAGG-1,-0.668048,-0.077593,1.199912,-0.403004,2.578630,-0.306615,3.107885,-0.103783,-0.567012,-0.080026,...,-0.25909,-0.568908,-0.439749,2.495847,-0.362863,-0.119038,-0.099579,-0.431536,-0.02837,-0.044405
TTGTTTCATTAGTCTA-1,2.214524,-0.077593,1.583931,-0.403004,1.881171,-0.306615,1.522866,-0.103783,-0.567012,-0.080026,...,-0.25909,2.141036,3.270130,2.058886,-0.362863,-0.119038,-0.099579,-0.431536,-0.02837,-0.044405
TTGTTTCCATACAACT-1,-0.668048,-0.077593,0.815200,-0.403004,1.683571,-0.306615,-0.552796,-0.103783,-0.567012,-0.080026,...,-0.25909,-0.568908,-0.439749,0.128602,-0.362863,-0.119038,-0.099579,-0.431536,-0.02837,-0.044405
TTGTTTGTATTACACG-1,-0.668048,-0.077593,0.812867,-0.403004,0.577428,-0.306615,-0.552796,-0.103783,-0.567012,-0.080026,...,-0.25909,-0.568908,-0.439749,0.630110,-0.362863,-0.119038,-0.099579,-0.431536,-0.02837,-0.044405
